<a href="https://colab.research.google.com/github/tefarov/netology_pyda5/blob/master/homework_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from datetime import datetime as dt
import sys

# Задание 1.
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId

In [0]:
STR_PTHFOLD = sys.path[0]
STR_PTHRATG = STR_PTHFOLD + r"\ratings.csv"

# импорт данных и превращаем timestamp  в читаемую datetime
rat_df = pd.read_csv(STR_PTHRATG)
rat_df['time'] = rat_df['timestamp'].apply(lambda x: dt.fromtimestamp(x))
rat_df.head()

In [0]:
# группируем рэйтинги по пользователю
usr_gp = rat_df.groupby('userId')
# создаём таблицу пользователей и отбираме тех,
# у которых количество оценок соответствует заданному порогу
usr_df = usr_gp.count()[['time']]
usr_df = usr_df[usr_df.time > 100]
usr_df.head()

In [0]:
min_df = usr_gp.min()[['time']] # фрейм с временем первой оценки
max_df = usr_gp.max()[['time']] # фрейм с врменеем последней оценки
# объединяем по индексу всё в один фрейм
usr_df = (usr_df
            .join(min_df,how='left',rsuffix='_min')
            .join(max_df,how='left',rsuffix='_max')
            )
usr_df.head()

In [0]:
# расчёт средней продолжительности жизни для каждого пользователя
usr_df['lifetime'] = (usr_df['time_max'] - usr_df['time_min'])
# расчёт средней продолжительности жизни по всем отобранным пользователям
lft = usr_df['lifetime'].mean()

print("Средняя продолжительность жизни пользователя: {0} дн. {1} ч. {2} мин. {3} сек.".format(lft.components.days,lft.components.hours,lft.components.minutes,lft.components.seconds))

# Задание 2
Дана статистика услуг перевозок клиентов компании по типам
rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

Необходимо сформировать две таблицы:
 1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
 2. аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [0]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [0]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [0]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [0]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

In [0]:
dat = (auto
    .merge(rzd,how='outer',on='client_id')
    .merge(air,how='outer',on='client_id')
)
dat

In [0]:
dat = (client_base
        .merge(rzd ,how='left',on='client_id')
        .merge(air ,how='left',on='client_id')
        .merge(auto,how='left',on='client_id')
    )
dat